In [ ]:
import random
import nltk
import numpy as np
import string
import sqlite3
import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from nltk.stem import WordNetLemmatizer

# Download required NLTK data
nltk.download('punkt')
nltk.download('wordnet')
# (Optional) Remove or comment out the following line if not needed:
# nltk.download('punkt_tab')

lemmatizer = WordNetLemmatizer()

# Expanded intents dataset with more examples per intent
intents = [
    {
        "tag": "greeting",
        "patterns": [
            "Hi", "Hello", "Hey", "How are you", "What's up", "Hi there", "Hello there"
        ],
        "responses": [
            "Hi there", "Hello", "Hey", "I'm fine, thank you", "Nothing much"
        ]
    },
    {
        "tag": "goodbye",
        "patterns": [
            "Bye", "See you later", "Goodbye", "Take care", "Farewell", "Catch you later"
        ],
        "responses": [
            "Goodbye", "See you later", "Take care", "Farewell!"
        ]
    },
    {
        "tag": "thanks",
        "patterns": [
            "Thank you", "Thanks", "Thanks a lot", "I appreciate it",
            "Much appreciated", "Thank you very much"
        ],
        "responses": [
            "You're welcome", "No problem", "Glad I could help", "Anytime!"
        ]
    },
    {
        "tag": "about",
        "patterns": [
            "What can you do", "Who are you", "What are you", "What is your purpose",
            "Tell me about yourself", "Describe yourself"
        ],
        "responses": [
            "I am a chatbot", "My purpose is to assist you",
            "I can answer questions and provide assistance",
            "I'm here to chat and help you out."
        ]
    },
    {
        "tag": "help",
        "patterns": [
            "Help", "I need help", "Can you help me", "What should I do",
            "Assist me", "I require assistance"
        ],
        "responses": [
            "Sure, what do you need help with?",
            "I'm here to help. What's the problem?",
            "How can I assist you?",
            "What do you need help with?"
        ]
    },
    {
        "tag": "age",
        "patterns": [
            "How old are you", "What's your age", "Tell me your age",
            "When were you created", "What is your age"
        ],
        "responses": [
            "I don't have an age. I'm a chatbot.",
            "I was just born in the digital world.",
            "Age is just a number for me.",
            "I don't age."
        ]
    },
    {
        "tag": "weather",
        "patterns": [
            "What's the weather like", "How's the weather today",
            "Tell me the weather", "Weather update please"
        ],
        "responses": [
            "I'm sorry, I cannot provide real-time weather information.",
            "You can check the weather on a weather app or website.",
            "I don't have current weather data."
        ]
    },
    {
        "tag": "budget",
        "patterns": [
            "How can I make a budget", "What's a good budgeting strategy",
            "How do I create a budget", "Help me plan my budget",
            "Give me budgeting tips"
        ],
        "responses": [
            "To make a budget, start by tracking your income and expenses. Then, allocate your income towards essential expenses like rent, food, and bills. Next, allocate some of your income towards savings and debt repayment. Finally, allocate the remainder of your income towards discretionary expenses like entertainment and hobbies."
        ]
    },
    {
        "tag": "credit_score",
        "patterns": [
            "What is a credit score", "How do I check my credit score",
            "How can I improve my credit score", "Tell me about credit scores",
            "Explain credit scores"
        ],
        "responses": [
            "A credit score is a number that represents your creditworthiness. It is based on your credit history and is used by lenders to determine whether or not to lend you money. The higher your credit score, the more likely you are to be approved for credit."
        ]
    },
    {
        "tag": "developer",
        "patterns": [
            "Who is your developer", "Who created you", "Who built you",
            "Who is your creator", "Tell me who made you", "Who made you"
        ],
        "responses": [
            "My developer is Suraj Sharma", "Suraj Sharma created me",
            "I was developed by Pavan", "My creator is Pavan"
        ]
    },
    {
        "tag": "play_music",
        "patterns": [
            "Hey can you play song for me", "please play the song for me",
            "play some music", "I want to listen to music", "play a song"
        ],
        "responses": [
            "Yes sure! Which song do you like to play?",
            "Yaa! Which song would you like?",
            "What song would you like to hear?",
            "Tell me the song name, and I'll play it."
        ]
    }
]

# Build training data
all_patterns = []
all_labels = []
responses = {}

for intent in intents:
    for pattern in intent['patterns']:
        all_patterns.append(pattern)
        all_labels.append(intent['tag'])
    responses[intent['tag']] = intent['responses']

def clean_up_sentence(sentence):
    """
    Removes punctuation, tokenizes, and lemmatizes the sentence.
    """
    sentence = sentence.translate(str.maketrans('', '', string.punctuation))
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

vectorizer = TfidfVectorizer(tokenizer=clean_up_sentence)
X = vectorizer.fit_transform(all_patterns)

classifier = LogisticRegression(max_iter=200)
classifier.fit(X, all_labels)

def get_response(user_input, threshold=0.1):
    """
    Predicts the intent of the user_input and returns an appropriate response.
    If the prediction confidence is below the threshold, returns a fallback message.
    """
    input_vector = vectorizer.transform([user_input])
    probabilities = classifier.predict_proba(input_vector)[0]
    max_prob = max(probabilities)

    print("Probabilities:", dict(zip(classifier.classes_, probabilities)))

    if max_prob < threshold:
        return "I'm sorry, I didn't quite understand that. Could you please rephrase?"
    predicted_tag = classifier.predict(input_vector)[0]
    response = random.choice(responses[predicted_tag])
    return response

# Database setup using SQLite
conn = sqlite3.connect("chatbot.db")
cursor = conn.cursor()
cursor.execute('''
    CREATE TABLE IF NOT EXISTS chat_logs (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        timestamp TEXT,
        user_message TEXT,
        bot_response TEXT
    )
''')
conn.commit()

def log_conversation(user_message, bot_response):
    """
    Inserts a record of the conversation into the chat_logs table.
    """
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    cursor.execute("INSERT INTO chat_logs (timestamp, user_message, bot_response) VALUES (?, ?, ?)",
                   (timestamp, user_message, bot_response))
    conn.commit()

def chat():
    """
    Starts an interactive chat with the user.
    """
    print("Chatbot: Hi, I'm your friendly chatbot. How can I help you today? (type 'quit' to exit)")
    while True:
        user_input = input("You: ")
        if user_input.lower() == "quit":
            print("Chatbot: Goodbye!")
            break
        response = get_response(user_input)
        print(f"Chatbot: {response}")
        log_conversation(user_input, response)

    # Close the database connection after the chat ends
    conn.close()

if __name__ == "__main__":
    chat()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Chatbot: Hi, I'm your friendly chatbot. How can I help you today? (type 'quit' to exit)
You: Who is your developer?
Probabilities: {'about': 0.0956813739946235, 'age': 0.0973226666716669, 'budget': 0.05571783913603745, 'credit_score': 0.057283248512274135, 'developer': 0.313167120473459, 'goodbye': 0.07280032083303765, 'greeting': 0.07642545106102801, 'help': 0.0631602222679161, 'play_music': 0.05469950955132566, 'thanks': 0.06793218259261435, 'weather': 0.04581006490601732}
Chatbot: My creator is Pavan
You: exit
Probabilities: {'about': 0.09310946764482914, 'age': 0.07901237871120928, 'budget': 0.08254981931747878, 'credit_score': 0.07192307866631122, 'developer': 0.08212425416932385, 'goodbye': 0.11626801267178354, 'greeting': 0.12481057091140464, 'help': 0.09789734271883194, 'play_music': 0.0805900315324238, 'thanks': 0.10668131529711929, 'weather': 0.06503372835928459}
Chatbot: Hi there
You: quit
Chatbot: Goodbye!
